# Web Scrapping in Python


**TODO pour moi avant le TME**

1.   Faire la version étudiant en enlevant les réponses. Ne garder qu'un code générique pour la carte qu'ils devront adapter



The first objective of this notebook is to discover the `request` and `BeautifulSoup` libraries to crawl a table on a Wikitable page, build a dataframe, and create a map.

*   `request` and [urllib](https://docs.python.org/3/library/urllib.html#module-urllib) for requestion REST API
*   [BeautifulSoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to inspect webpages

Please note that this part serves as an initiation of web scrapping but you will need to learn by yourself to make the project. It is inspired from notebooks published by Galiana, Lino. 2023. Python Pour La Data Science. https://doi.org/10.5281/zenodo.8229676. Exercices target different sources of information, the code needs to be adapted.











In [5]:
!pip install -q lxml

import bs4
import lxml
import pandas as pd
import urllib
import numpy as np

from urllib import request

## Exercice 1 : Scrap a table on a wikipedia page

We would like to display on a map the location of Summer Olympic Games since 1896. We will use a [Wikipedia page](https://fr.wikipedia.org/wiki/Jeux_olympiques) to scrap the associated table.
Below is the code to extract the content of the page using `request` and display its title using `BeautifulSoup`.

In [9]:
jo = "https://fr.wikipedia.org/wiki/Jeux_olympiques"

request_text = request.urlopen(jo).read()
print(request_text[:1000])
page = bs4.BeautifulSoup(request_text, "lxml")
print(page.find("title"))

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" lang="fr" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Jeux olympiques \xe2\x80\x94 Wikip\xc3\xa9dia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 ve

Our objective here is to extract the different information in the first table "Jeux olympiques d'été" and to build a data frame.

To proceed, you will have to follow these steps:
*   Find the table
*   Collect each line of the table
*   Retrieve the different columns and transform them into text format. Also, use `strip` to format the value into a proper text format (e.g., without useless spaces). Store these lines (formated as a table of columns) in a table.

**Warning: We have two location for 1940. You have therefore to manage this case: identify the attribute indicating this fact and write a code to collect those lines and insert in the previous table in the right place**

* Collect headers of the HTML table
* Build a data frame from the result table and the headers








In [10]:
#[student]

page = bs4.BeautifulSoup(request_text, "lxml")
table_found = str(page.find("table", {"class" : "wikitable center" }))
table = bs4.BeautifulSoup(table_found, "lxml")
lines = table.findAll("tr")

collumn_names = []
data = []

names = lines[0].findAll("th")

collumn_names = [th.get_text(strip=True) for th in names]
val = 0
for i in range(1, len(lines)):

  n_line = lines[i].findAll("td")
  val -= 1
  # Vérifier si on a plusieurs villes/pays/contients
  if(n_line[0].has_attr("rowspan")):
    tmp_info_line = [td.get_text(strip=True) for td in n_line]
    data.append(tmp_info_line)

    val = int(n_line[0]['rowspan'])

    # Recupérer les autres villes/pays/contients
    for next in range(1, val):

      line = [tmp_info_line[0],tmp_info_line[1]]

      for td in lines[i+next]:
        td_string = td.get_text(strip=True)
        if(td_string):
          line.append(td_string)
      print("line : ", line)
      data.append(line)
      
  elif val <= 0:
    data.append([td.get_text(strip=True) for td in n_line])


data = np.asarray(data[:-1])
df = pd.DataFrame(data[:-1], columns=collumn_names)
print(df)



line :  ['1940', 'XII(annulés[NB 4])', 'Helsinki', 'Finlande', 'Europe']
   Année Olympiade(Nº de l’olympiade[NB 1])         Ville hôte  \
0   1896                                  I         Athènes(1)   
1   1900                                 II           Paris(1)   
2   1904                                III     Saint-Louis(1)   
3   1906             •(Intercalaires[NB 2])         Athènes(2)   
4   1908                                 IV         Londres(1)   
5   1912                                  V       Stockholm(1)   
6   1916                  VI(annulés[NB 3])             Berlin   
7   1920                                VII          Anvers(1)   
8   1924                               VIII           Paris(2)   
9   1928                                 IX       Amsterdam(1)   
10  1932                                  X     Los Angeles(1)   
11  1936                                 XI          Berlin(1)   
12  1940                 XII(annulés[NB 4])              Tokyo   
13 

## Exercice 2: Locate places on a map

The objective here is to identify organizer cities on a map.
You will have to code the following steps:
* Collect the URL of each city
* Go to this page using `urllib.request`
* Find the coordinates
* Store cities and coordinates in a data frame

In [1]:
#[student]

We then transform degree coordinates into numerical coordinates to build a map


In [16]:
import re

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S', 'O'):
        dd *= -1
    return dd

def parse_dms(dms):
    parts = re.split('[^\d\w]+', dms)
    #potentially needs an adaptation
    #[student]
    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])
    #lng = dms2dd(parts[4], parts[5], parts[6], parts[7])
    return lat

In [110]:
import urllib.parse

def transform_wikipedia_url(wiki_path):
    # Decode the URL path
    decoded_path = urllib.parse.unquote(wiki_path)
    
    # Check if the path starts with "/wiki/"
    if decoded_path.startswith("/wiki/"):
        # Extract the article title
        article_title = decoded_path[len("/wiki/"):]
        
        # Encode the non-ASCII characters in the article title
        encoded_title = urllib.parse.quote(article_title)
        
        # Construct the transformed URL
        transformed_url = f"https://fr.wikipedia.org/wiki/{encoded_title}"
        
        return transformed_url
    else:
        # Return the original path if it doesn't start with "/wiki/"
        return wiki_path

In [116]:
data_coordinates = []

page = bs4.BeautifulSoup(request_text, "lxml")
table_found = str(page.find("table", {"class" : "wikitable center" }))

table = bs4.BeautifulSoup(table_found, "lxml")
lines = table.findAll("tr")

hrefs = [] 
for i in range(1, len(lines)):

  n_line = lines[i].findAll("td")

  if len(n_line) >= 3:
      # Récupérer la balise de la ville
      td_city = n_line[2]
      td = bs4.BeautifulSoup(str(td_city), "lxml")

      # Récupérer les balises a de la ligne
      a_tag_list = td.find("a")

      if(a_tag_list):
        # Récupérer la ref de la page wikepedia de la ville 
        href = a_tag_list.get('href')

        # Récupérer l'url de la ville
        url = transform_wikipedia_url(href)
        response = urllib.request.urlopen(url)

        # Lire le contenu de la page web de la ville
        html_content = response.read()



#data['latitude'] = data['latitude'].apply(parse_dms)
##data['longitude'] = data['longitude'].apply(parse_dms)
# print(data)

The map can be obtain with the following code

In [14]:
import geopandas as gpd
from pathlib import Path
import folium
from IPython.display import display

gdf = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.longitude, data.latitude))

Path("leaflet").mkdir(parents=True, exist_ok=True)

center = gdf[['latitude', 'longitude']].mean().values.tolist()
sw = gdf[['latitude', 'longitude']].min().values.tolist()
ne = gdf[['latitude', 'longitude']].max().values.tolist()

m = folium.Map(location = center, tiles='openstreetmap')

# I can add marker one by one on the map
for i in range(0,len(gdf)):
    folium.Marker([gdf.iloc[i]['latitude'], gdf.iloc[i]['longitude']], popup=gdf.iloc[i]['name']).add_to(m)

m.fit_bounds([sw, ne])
display(m)

ModuleNotFoundError: No module named 'geopandas'

# Project

The objective of this project is to practice all concepts taught in the main lecture. Therefore, you will have to collect data around a thematic, identify a problematic, clean and format the data, provide some exploratory analysis and visualization, build models and evaluate them. You will have also to design dashboard and to storytell the whole pipeline.


The whole project will have two outputs describing the methodology and the results:
* A technical report targeting your datascientist colleagues
* An oral presentation targeting your CEO, chief, client. In this case, we assume that the audience is not specialized in data science. (But you also need to present the methodology to convince them to trust the results).


**Requests:**
* Team work of two people (same group throughout the semester)
* All your work should be stored on a git repo: [tutorial](https://github.com/baskiotisn/2IN013robot2023/blob/d979333fb80c9b6acd9515aaec040943d10d365c/docs/tutoriel_git.pdf)

**Remarks:**
There are also other libraries or issues you'll encounter when collecting data. Some of these are listed below, but don't be shy and interact with a search engine to solve your own issue!
* [Regular expressions](https://docs.python.org/3/howto/regex.html) might be useful!
* [API with authentication](https://www.geeksforgeeks.org/authentication-using-python-requests/)
*   [Selenium](https://selenium-python.readthedocs.io/) when pages are generated wia javascript scripts
* [Playwright](https://playwright.dev/) --> looks easier and more adapted than Selenium
* [Scrapy](https://scrapy.org/) for web crawling / or when you don't know the URL.[Tuto here](https://doc.scrapy.org/en/latest/intro/tutorial.html)
* [Summary of some difficulties in scrapping data from the web](https://www.zenrows.com/blog/web-scraping-challenges#page-structure-changes)

## Your daily task
Collecting data from the web and open data portals. Open data might include csv files, you can use them, but be aware that scraping should be your main acitivity in the dataset gathering.
You can begin to format the data, merge them to build dataframe that would be analyzed next week.